# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902063


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:03<00:57,  1.97s/it]

Rendering models:  10%|▉         | 3/31 [00:04<00:42,  1.53s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:43,  1.67s/it]

Rendering models:  19%|█▉        | 6/31 [00:08<00:31,  1.26s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:25<00:02,  1.29it/s]

Rendering models:  94%|█████████▎| 29/31 [00:25<00:01,  1.43it/s]

Rendering models:  97%|█████████▋| 30/31 [00:26<00:00,  1.16it/s]

Rendering models: 100%|██████████| 31/31 [00:27<00:00,  1.31it/s]

equidad1                              0.000052
JessieDawn                            0.000130
Thedunkmasta                          0.001058
mlang2017                             0.000112
Alexander00                           0.000034
not-logged-in-ab1612bc18cd7aab85db    0.000037
not-logged-in-769e51a01946aca9871e    0.012210
cedar-phoenix                         0.000030
minichick                             0.000018
Linda_J._Berkel                       0.000039
browny811                             0.000028
Jessica_Korkmaz                       0.004103
not-logged-in-d243acae43f6f5b362ef    0.000909
not-logged-in-5f30b2bb9b7f9d49da41    0.000102
not-logged-in-09330e0b15263888ee9d    0.000015
clairedeu                             0.000037
not-logged-in-4cf75754fa9abc7408cc    0.000111
not-logged-in-4c6d97421fdb5b2fcda3    0.000080
Lavadude                              0.015591
not-logged-in-bc61a136736b0aa8edfc    0.006111
ts135992                              0.000027
w7250369     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())